In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="GYyB5SrSjvJ2EZsyJgCl")
project = rf.workspace("yolo-boyoh").project("blood-postion-t936i")
version = project.version(1)
dataset = version.download("folder")
                

  Using cached roboflow-1.2.1-py3-none-any.whl (86 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached certifi-2025.7.14-py3-none-any.whl (162 kB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
  Using cached pillow_heif-1.0.0-cp39-cp39-win_amd64.whl (5.3 MB)
  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached pillow-11.3.0-cp39-cp39-win_amd64.whl (7.0 MB)
  Using cached matplotlib-3.9.4-cp39-cp39-win_amd64.whl (7.8 MB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Using cached kiwisolver-1.4.7-cp39-cp39-win_amd64.whl (55 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl (162 kB)
  Using cached idna-3.7-py3-none-any.whl (66 kB)
  Using cached pillow_avif

You should consider upgrading via the 'D:\v-intership part\Blood\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to blood-postion-1 in folder:: 100%|██████████| 99/99 [00:00<00:00, 1119.73it/s]


In [2]:
%pip install tensorflow

  Using cached tensorflow-2.19.0-cp39-cp39-win_amd64.whl (375.7 MB)
  Using cached termcolor-3.1.0-py3-none-any.whl (7.7 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached wrapt-1.17.2-cp39-cp39-win_amd64.whl (38 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
  Using cached h5py-3.14.0-cp39-cp39-win_amd64.whl (2.9 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.6.0-py3-none-any.whl (21 kB)
  Using cached grpcio-1.73.1-cp39-cp39-win_amd64.whl (4.3 MB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
  Using cached protobuf-5.29.5-cp39-cp39-win_amd64.whl (434 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
  Using cached ml_dtypes-0.5.1-cp39-cp39-win_amd64.whl (209 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
  Usin

You should consider upgrading via the 'd:\v-intership part\Blood\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import os

In [4]:
base_dir = dataset.location
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'valid')

In [6]:
IMG_SIZE = 224
BATCH_SIZE = 32

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)


In [8]:
train_gen = train_datagen.flow_from_directory(train_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical')
val_gen = val_datagen.flow_from_directory(val_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical')

Found 78 images belonging to 3 classes.
Found 4 images belonging to 3 classes.


In [9]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # freeze base

In [10]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=preds)


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
%pip install scipy

# Train
model.fit(train_gen, validation_data=val_gen, epochs=200)

You should consider upgrading via the 'd:\v-intership part\Blood\myenv\Scripts\python.exe -m pip install --upgrade pip' command.



  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)
Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7436 - loss: 0.8438

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.


3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7660 - loss: 0.7903 - val_accuracy: 0.7500 - val_loss: 0.9023
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9493 - loss: 0.1401 - val_accuracy: 0.7500 - val_loss: 0.4052
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 929ms/step - accuracy: 0.9819 - loss: 0.0446 - val_accuracy: 1.0000 - val_loss: 0.0956
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 952ms/step - accuracy: 1.0000 - loss: 0.0042 - val_accuracy: 1.0000 - val_loss: 0.0296
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 924ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 0.0121
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 1.0000 - val_loss: 0.0064
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 967ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 0.0043
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 933ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 0.0033
Epoch

In [14]:
model.save("mobilenetv2_lab_instruments.h5")